In [1]:
import torch
import sys
from torch.utils.data import DataLoader
import torchvision.transforms as T
from tqdm import tqdm
import numpy as np
from mean_average_precision import MetricBuilder

sys.path.append('../src/')
from newspapersdataset import (
    NewspapersDataset, 
    prepare_data_for_dataloader
)
from functions import (
    from_tsv_to_list,
    collate_fn
)

sys.path.append("../../")
from image_size import get_image_size  # source: https://github.com/scardine/image_size

In [2]:
model = torch.load('../saved_models/model.pth', map_location=torch.device('cpu'))

In [3]:
parameters = {
    'channel': 1, 
    'rescale': [1000, 1000],
    'batch_size': 16,
    'shuffle': False, 
    'num_workers': 2,
    'main_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/Master_degree/',
    'image_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/scraped_photos_final/',
    'annotations_dir': '/Users/alexdrozdz/Desktop/Studia/00. Seminarium magisterskie/news-navigator/',
}

In [4]:
data_transform = T.Compose([
    T.Grayscale(num_output_channels=parameters['channel']),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,)),
    ])

In [5]:
# prepare data 
expected = from_tsv_to_list(parameters['annotations_dir']+'test-A/expected.tsv')
in_file = from_tsv_to_list(parameters['annotations_dir']+'test-A/in.tsv')
img_paths = [parameters['image_dir']+path for path in in_file]

data = prepare_data_for_dataloader(
    img_dir=parameters['image_dir'],
    in_list=in_file,
    expected_list=expected,
    bbox_format='x0y0x1y1',
    scale=parameters['rescale'],
    test=False,
    )
dataset = NewspapersDataset(
    df=data,
    images_path=img_paths,
    scale=parameters['rescale'],
    transforms=data_transform,
    test=False,
    )
dataloader = DataLoader(
    dataset,
    batch_size=parameters['batch_size'],
    shuffle=parameters['shuffle'],
    collate_fn=collate_fn,
    num_workers=parameters['num_workers'],
    )

In [6]:
# predict on cpu
device = 'cpu'
model.to(device)
torch.set_num_threads(1)
cpu_device = torch.device('cpu')

In [7]:
# predict on the test set
with torch.no_grad():
    f_out, f_tar = [], []
    for images, targets in tqdm(dataloader):
        images = list(img.to(cpu_device) for img in images)
        targets = [{k: v.to(cpu_device) for k, v in t.items()} for t in targets]
        f_tar.append(targets)
        outputs = model(images)
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        f_out.append(outputs)

100%|██████████| 14/14 [20:48<00:00, 89.17s/it]


In [135]:
f_out_flat = [x for xs in f_out for x in xs]
f_tar_flat = [x for xs in f_tar for x in xs]

In [136]:
# prepare results
pred_list, gt_list = [], []
for i in range(len(f_out_flat)):
    # prediction
    temp_pred = []
    for ii_pred in range(len(f_out_flat[i]['boxes'].detach().numpy())):
        obj_pred = [int(el) for el in f_out_flat[i]['boxes'].detach().numpy()[ii_pred]]
        obj_pred.append(f_out_flat[i]['labels'].detach().numpy()[ii_pred]-1)
        obj_pred.append(f_out_flat[i]['scores'].detach().numpy()[ii_pred])
        temp_pred.append(obj_pred)
    pred_list.append(np.array(temp_pred))
    # ground truth
    temp_gt = []
    for ii_gt in range(len(f_tar_flat[i]['boxes'].detach().numpy())):
        obj_gt = [int(el) for el in f_tar_flat[i]['boxes'].detach().numpy()[ii_gt]]
        obj_gt.append(f_tar_flat[i]['labels'].detach().numpy()[ii_gt]-1)
        obj_gt = obj_gt + [0, 0]
        temp_gt.append(np.array(obj_gt))
    gt_list.append(np.array(temp_gt))

In [137]:
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=True, num_classes=7)

for i in range(len(pred_list)):
    metric_fn.add(pred_list[i], gt_list[i])

In [138]:
metric = metric_fn.value(iou_thresholds=np.arange(0.5, 1.0, 0.05), recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy='soft')

In [139]:
# final results
ap_dict = {
    'photograph': metric[0.5][0]['ap'],
    'illustration': metric[0.5][1]['ap'],
    'map': metric[0.5][2]['ap'],
    'cartoon': metric[0.5][3]['ap'],
    'editorial_cartoon': metric[0.5][4]['ap'],
    'headline': metric[0.5][5]['ap'],
    'advertisement': metric[0.5][6]['ap'],
    'mAP': metric['mAP']
}

In [140]:
ap_dict

{'photograph': 0.76625425,
 'illustration': 0.31763086,
 'map': 0.65815836,
 'cartoon': 0.67715204,
 'editorial_cartoon': 0.3822639,
 'headline': 0.9000255,
 'advertisement': 0.81451476,
 'mAP': 0.5231611}